In [2]:
# Mounting Google Drive (Opsional, tapi disarankan jika gambar disimpan di Drive)
from google.colab import drive
drive.mount('/content/drive')

# Import pustaka utama
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image
import time
import functools
import os

def tensor_to_image(tensor):
    """Mengubah tensor TensorFlow kembali menjadi gambar standar."""

    # Mengubah nilai tensor dari [0, 1] menjadi [0, 255]
    tensor = tensor*255
    # Mengubah tipe data menjadi integer 8-bit (uint8)
    tensor = np.array(tensor, dtype=np.uint8)

    # Menghapus dimensi batch jika ada
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]

    return PIL.Image.fromarray(tensor)


def load_img(path_to_img):
    """Memuat gambar dan mengubah ukurannya agar kompatibel dengan model (max_dim=512)."""

    max_dim = 512

    # Memuat gambar mentah
    img = tf.io.read_file(path_to_img)
    # Mendekode gambar menjadi tensor dengan 3 channel (RGB)
    img = tf.image.decode_image(img, channels=3)
    # Mengubah tipe data menjadi float32 (normalisasi ke [0, 1])
    img = tf.image.convert_image_dtype(img, tf.float32)

    # Perhitungan untuk resize
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim/long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    # Resize gambar
    img = tf.image.resize(img, new_shape)

    # Menambahkan dimensi batch (dimensi [1, height, width, 3])
    img = img[tf.newaxis,:]

    return img

print("Memuat gambar...")

# ⚠️ GANTI PATH INI DENGAN PATH GAMBAR ANDA DI DRIVE/COLAB ⚠️
content_path = '/content/Bunga 🌹.jpeg'
style_path = '/content/Buket Kosong.jpeg'

# Cek keberadaan file dan mengunduh sampel jika tidak ada
if not os.path.exists(content_path) or not os.path.exists(style_path):
    print("File 'singa.jpg' atau 'Lukisan Cat Air.jpg' tidak ditemukan.")
    print("Mengunduh gambar sampel dari internet untuk demonstrasi...")

    # Unduh gambar konten (anjing Labrador)
    content_path = tf.keras.utils.get_file('content.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
    # Unduh gambar gaya (lukisan Kandinsky)
    style_path = tf.keras.utils.get_file('style.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky_-_Composition_7.jpg')

# Memuat gambar sebagai tensor
content_image = load_img(content_path)
style_image = load_img(style_path)

print("Gambar berhasil dimuat!")

# Memuat Model Neural Style Transfer
print("Sedang memuat model Neural Style Transfer dari TensorFlow Hub...")
# Model yang digunakan adalah Arbitrary Image Stylization (Fast and Artistic)
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("Model berhasil dimuat.")

# Proses Style Transfer
print("Sedang memproses kombinasi gambar (ini mungkin memakan waktu beberapa detik)...")

start_time = time.time()

# Melakukan style transfer
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

end_time = time.time()

print(f"Selesai dalam {end_time - start_time:.2f} detik.")

# Menyimpan Hasil
output_filename = 'hasil_kombinasi.jpg'

# Konversi tensor hasil ke format gambar PIL
final_image = tensor_to_image(stylized_image)

# Simpan gambar
final_image.save(output_filename)

print(f"Gambar hasil kombinasi telah disimpan sebagai: {output_filename}")
print("Silakan unduh gambar tersebut dari panel file di sebelah kiri.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memuat gambar...
Gambar berhasil dimuat!
Sedang memuat model Neural Style Transfer dari TensorFlow Hub...
Model berhasil dimuat.
Sedang memproses kombinasi gambar (ini mungkin memakan waktu beberapa detik)...
Selesai dalam 4.93 detik.
Gambar hasil kombinasi telah disimpan sebagai: hasil_kombinasi.jpg
Silakan unduh gambar tersebut dari panel file di sebelah kiri.
